<a href="assets/img/grass_and_satellites.png">
  <img src="assets/img/grass_and_satellites.png"
   width="25%" style="float:right">
</a>

# **<span style='color:Green'>GRASS GIS for remote sensing data processing and analysis</span>**

### Workshop at FOSS4G 2022, Florence (Italy)

*Lecturers:* Veronica Andreo, Markus Neteler & Maris Nartiss

*Date:* 2022-08-23


### Foreword

This notebook will demonstrate the use of **GRASS GIS 8.2+** in combination with Python within a Jupyter Notebook environment. We will use GRASS modules and related python libraries that facilitate scripting (`grass.script`) and connection/interaction with Jupyter Notebooks (`grass.jupyter`). 

The workflow that will be demonstrated on this notebook ranges from searching satellite data to time series building and supervised classification.

### Dependencies

Some Python dependencies are needed to run the proposed exercises of this notebook. Install as follows:

In [ ]:
!pip3 install sentinelsat notebook folium scikit-learn pandas numpy seaborn matplotlib ipywidgets

### Table of contents

1. Why Jupyter Notebooks and how to use them?
2. GRASS GIS basics
3. GRASS GIS & Python
4. Getting ready: set variables and download sample data
5. Initialization of GRASS GIS in the Jupyter notebook session
6. Creating an area of interest map
7. Importing geodata into GRASS GIS
8. Sentinel-2 processing overview
9. Computing NDVI
10. Time series data processing
11. Creating an image stack (imagery group)
12. Object recognition with image segmentation
13. Supervised Classification: Random Forest
14. Supervised Classification: Maximum Likelihood and band references
15. What's next?

## 1. Why Jupyter Notebooks and how to use them?

Jupyter Notebooks are server-client applications that allow code written in a notebook document to be
**edited and executed through a web browser**. They can be run on a local computer (no internet access required) or used to control computations on a remote server accessed via the Internet
([see the documentation](https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)).

Jupyter Notebooks can be interactive and they allow to combine live code, explanatory text, and computational results in a single document. In general, they are:

* convenient for initial code development (prototyping)
* ideal for code segmentation with the ability to re-run cells
* able to store values of variables from already executed cells

The notebook can be saved as an executable Python script in addition to the native `.ipynb` format, 
or exported to various documentation formats such as PDF or Sphinx RST with nice styling.

#### Editing and interactive use

Editing a Jupyter Notebook is very easy: in the web browser, you can navigate between text or code
cells using the mouse or keyboard shortcuts (see Menu > Help > Keyboard Shortcuts). You can
execute small code chunks cell by cell, save the notebook in its current state, or modify and 
recalculate cells or return them to their previous state. In addition to executable code cells, you 
can use Markdown in documentation cells to make them presentable to others.

## 2. GRASS GIS basics


### Open GRASS for the first time

As of version 8.0, GRASS has modified its startup to make it more user friendly:
<br>
<a href="assets/img/grass_start.png">
  <img src="assets/img/grass_start.png"
   alt="First time launching GRASS 8"
   title="First time launching GRASS 8"
   width="65%">
</a>

From the **Data** catalog tab you can manage several actions and if you do not yet have imported data into the GRASS database, the software creates the directory structure or database automatically.

### Database

- **GRASS database** (directory with projects): When running GRASS GIS for the first time, a folder named "grassdata" is automatically created. Depending on the operating system, it can be found in `$HOME` (*nix) or `My Documents` (MS Windows).
- **Location** (a project): A location is defined by its coordinate reference system (CRS). The location that is automatically created is in WGS84 (EPSG:4326). If you have data in another CRS, you should ideally create a new location.
- **Mapset** (a subproject): Each location can have many mapsets to manage different aspects or sub-regions of a project. When creating a new location, GRASS GIS automatically creates a special mapset called *PERMANENT* where the central data of the project (e.g., base maps, road network, dem, etc.) can be stored. 

<a href="assets/img/grass_database.png">
  <img src="assets/img/grass_database.png"
   alt="GRASS GIS database"
   title="GRASS GIS database"
   width="50%">
</a>

<div class="alert alert-info">More info: <a href="https://grass.osgeo.org/grass-stable/manuals/grass_database.html">https://grass.osgeo.org/grass-stable/manuals/grass_database.html</a>.</div>

### Computational region

Another fundamental concept of GRASS GIS (and very useful when working with raster data) is that of the **computational region**. It refers to the boundary configuration of the analysis area and spatial resolution (raster). The **computational region** can be defined and modified with the command [g.region](https://grass.osgeo.org/grass-stable/manuals/g.region.html) to the extent of a vector map, a raster or manually to some area of interest. The *output raster maps will have an extent and spatial resolution equal to the computational region*, while vector maps are always processed at their original extent.

<a href="assets/img/region.png">
  <img src="assets/img/region.png"
   alt="Computational region"
   title="Computational region"
   width="50%">
</a>

<div class="alert alert-info">For more details, see the wiki on <a href="https://grasswiki.osgeo.org/wiki/Computational_region">Computational Region</a>.

### Modules and extensions

GRASS has more than [500 modules](https://grass.osgeo.org/grass-stable/manuals/full_index.html) for the most varied tasks:

| Prefix                                                               | Function class   | Type of command                     | Example
|--------------------------------------------------------------------- |:---------------- |:----------------------------------- |:-------------------------------------------------------------------------------------------------------------------
| [g.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#g)    | general          | general data management             | [g.rename](https://grass.osgeo.org/grass-stable/manuals/g.rename.html): renames map
| [d.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#d)    | display          | graphical output                    | [d.rast](https://grass.osgeo.org/grass-stable/manuals/d.rast.html): display raster map 
| [r.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#r)    | raster           | raster processing                   | [r.mapcalc](https://grass.osgeo.org/grass-stable/manuals/r.mapcalc.html): map algebra
| [v.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#r)    | vector           | vector processing                   | [v.clean](https://grass.osgeo.org/grass-stable/manuals/v.clean.html): topological cleaning
| [i.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#i)    | imagery          | imagery processing                  | [i.pca](https://grass.osgeo.org/grass-stable/manuals/i.pca.html): Principal Components Analysis on imagery group
| [r3.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#r3)  | voxel            | 3D raster processing                | [r3.stats](https://grass.osgeo.org/grass-stable/manuals/r3.stats.html): voxel statistics
| [db.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#db)  | database         | database management                 | [db.select](https://grass.osgeo.org/grass-stable/manuals/db.select.html): select value(s) from table
| [ps.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#ps)  | postscript       | PostScript map creation             | [ps.map](https://grass.osgeo.org/grass-stable/manuals/ps.map.html): PostScript map creation
| [t.\*](https://grass.osgeo.org/grass-stable/manuals/full_index.html#t)    | temporal         | space-time datasets                 | [t.rast.aggregate](https://grass.osgeo.org/grass-stable/manuals/t.rast.aggregate.html): raster time series aggregation

Extensions or **add-ons** can be installed from the
[central GitHub repository](https://grass.osgeo.org/grass-stable/manuals/addons/) 
or from *other users' GitHub* (or similar repositories) using the command 
[g.extension](https://grass.osgeo.org/grass-stable/manuals/g.extension.html). For example:

```bash
 # install an extension from the GRASS GIS repository
 g.extension extension=r.hants
 
 # install an extension from another GitHub repository
 g.extension extension=r.change.stats \
   url=https://github.com/mundialis/r.change.stats
``` 

## 3. GRASS & Python

### Python package `grass.script`

The **grass.script** or GRASS GIS Python Scripting Library provides functions for calling GRASS modules within Python scripts. The most commonly used functions include:

- `run_command`: used when the output of the modules is a raster or vector, no text type output is expected
- `read_command`: used when the output of the modules is of text type
- `parse_command`: used with modules whose output can be converted to `key=value` pairs
- `write_command`: used with modules that expect text input, either in the form of a file or from stdin

It also provides several wrapper functions for frequently used modules, for example:

- To get info from a raster, script.raster.raster_info() is used: `gs.raster_info('dsm')`
- To get info of a vector, script.vector.vector_info() is used: `gs.vector_info('roads')`
- To list the raster in a location, script.core.list_grouped() is used: `gs.list_grouped(type=['raster'])`
- To obtain the computational region, script.core.region() is used: `gs.region()`

<div class="alert alert-info">More info: <a href="https://grass.osgeo.org/grass-stable/manuals/libpython/script_intro.html">https://grass.osgeo.org/grass-stable/manuals/libpython/script_intro.html</a>

### Python package `grass.jupyter`

The **grass.jupyter** library improves the integration of GRASS and Jupyter, and provides different classes to facilitate GRASS maps visualization:

- `init`: starts a GRASS session and sets up all necessary environment variables
- `Map`: 2D rendering
- `Map3D`: 3D rendering
- `InteractiveMap`: interactive visualization with folium
- `TimeSeriesMap`: visualization for spatio-temporal data

<div class="alert alert-info">More info: <a href="https://grass.osgeo.org/grass-stable/manuals/libpython/grass.jupyter.html">https://grass.osgeo.org/grass-stable/manuals/libpython/grass.jupyter.html</a>

## 4. Getting ready: set variables and download sample data

For the ease of working in this notebook, we define some session variables.

In [ ]:
import os

# data directory
homedir = os.path.join(os.path.expanduser('~'), "foss4g_grass4rs")

# GRASS GIS database variables
#grassbin = "grassdev"
grassbin = "grass"
grassdata = os.path.join(homedir, "grassdata")
location = "nc_spm_08_grass7"
mapset = "PERMANENT"

In [ ]:
# Sentinel-2 related directories
s2_data = os.path.join(homedir, "sentinel")
s2_timestamps = os.path.join(homedir, s2_data, "sentinel-timestamps.txt")

# create directories if not already existing
os.makedirs(grassdata, exist_ok=True)
os.makedirs(s2_data, exist_ok=True)

# the variables are also accessible via Python
print(homedir)

# list content
os.listdir(homedir)

Next we check the GRASS GIS installation:

In [ ]:
import subprocess
print(subprocess.check_output([grassbin, "--config", "version"], text=True))

Next, if not already there, we download **North Carolina location** and unpack it within the above defined `homedir`.

In [ ]:
# download NC sample data into target directory homedir (we use `wget` on command line here)
!wget -c https://grass.osgeo.org/sampledata/north_carolina/nc_spm_08_grass7.zip -O $homedir/nc.zip

In [ ]:
# unpack sample dataset into target directory homedir
!unzip -o -q -d $grassdata $homedir/nc.zip

print("List uploaded file(s) in target directory "+homedir+":")
os.listdir(homedir)

Download and unzip Sentinel-2 scenes:

In [ ]:
# download Sentinel-2 data into target directory homedir
!wget -c https://data.neteler.org/foss4g2022/sentinel.zip -O $homedir/sentinel.zip

# unpack into target directory homedir
!unzip -o -q -d $homedir $homedir/sentinel.zip

Download landuse map:

In [ ]:
# get NC landuse map 2019 in GRASS GIS format, to be used later as classification training map
!wget -c https://data.neteler.org/foss4g2022/nc_nlcd2019.pack -O $homedir/nc_nlcd2019.pack

In [ ]:
print("List uploaded file(s) in target directory "+homedir+":")
os.listdir(homedir)

## 5. Imports and initialization of GRASS GIS

In [ ]:
# Import standard Python packages we need
import sys

# Ask GRASS GIS where its Python packages are to be able to run it from the notebook
sys.path.append(
    subprocess.check_output([grassbin, "--config", "python_path"], text=True).strip()
)

In [ ]:
# Import the GRASS GIS packages we need
import grass.script as gs
import grass.jupyter as gj

# Start the GRASS GIS Session
session = gj.init(grassdata, location, mapset)

In [ ]:
# Show current GRASS GIS settings, this also checks if the session works
gs.gisenv()

Before we start, we list the elements in the mapset `PERMANENT`. If you only want to see the raster or vector type elements, just change the `type` option in the following command. As you can see, since the output is of text type, we use `read_command()`.

In [ ]:
# List vector elements in the PERMANENT mapset
gs.list_grouped(type="vector")

Now we import the landuse map into the PERMANENT mapset (so it is visible in all other mapsets).

In [ ]:
## Import of the North Carolina NLCD2019 raster map (subset; resampled to 10m)
gs.run_command("r.unpack", input=os.path.join(homedir, "nc_nlcd2019.pack"))

In [ ]:
# show metadata
print(gs.read_command("r.report", map="nc_nlcd2019"))

Next, we create a new mapset or project, where we will work with this notebook and import Sentinel-2 data.

In [ ]:
# Create a new mapset and switch to it
gs.run_command("g.mapset", mapset="sentinel2", flags="c")

In [ ]:
# Another way would be to use the grass.grassdb functions:

# import grass.grassdb.create as gsdb
# gsdb.create_mapset(grassdata, location, "sentinel2")
# session.switch_mapset("sentinel2")

In [ ]:
# Check current mapset
print(gs.read_command("g.mapset", flags="p"))

In [ ]:
# Print accessible mapsets within the location
print(gs.read_command("g.mapsets", flags="p"))

## 6. Creating an area of interest map

To search for Sentinel 2 images, we need an area of interest. This area can be defined by a vector map or the computational region. Here, will use a map of urban areas that we already have in the PERMANENT mapset to define the region. Since we are interested in the city of Raleigh, we use the function [v.extract](https://grass.osgeo.org/grass-stable/manuals/v.extract.html) to create a new polygon corresponding to that urban area only. Note that in this case we use `run_command()`.

In [ ]:
# Check `urbanarea` vector attributes
gs.vector_db_select('urbanarea')['values']

In [ ]:
# Extract Raleigh urban area from `urbanarea` vector map
gs.run_command("v.extract", 
               input="urbanarea", 
               where="NAME == 'Raleigh'", 
               output="urban_area_raleigh")

In [ ]:
# show attributes
gs.vector_db_select('urban_area_raleigh')['values']

We set the computational region to the boundaries of the newly created vector. This will be the bounding box we'll use for the Sentinel scenes search.

In [ ]:
# Set the computational region to the extent of Cordoba urban area
region = gs.parse_command("g.region", vector="urban_area_raleigh", flags="g")
region

We now use the `grass.jupyter` functions to display the newly obtained vector over the OpenStreetMap basemap.

In [ ]:
# Display newly created vector
raleigh_map = gj.InteractiveMap(width = 500, use_region=True, tiles="OpenStreetMap")
raleigh_map.add_vector("urban_area_raleigh")
raleigh_map.add_layer_control(position = "bottomright")
raleigh_map.show()

## 7. Sentinel-2 processing overview

There are plenty of libraries or tools which allow downloading
Sentinel products from [Copernicus Open Access Hub](https://scihub.copernicus.eu/).

For GRASS GIS there is the [i.sentinel](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.html) toolbox that facilitates searching, filtering, downloading, importing and pre-processing Sentinel data, especially Sentinel 2, from a GRASS GIS session. The toolbox consists of six GRASS addon modules:

* [i.sentinel.download](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.download.html)
* [i.sentinel.import](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.import.html)
* [i.sentinel.preproc](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.preproc.html)
* [i.sentinel.mask](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.mask.html)
* [i.sentinel.coverage](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.coverage.html)
* [i.sentinel.parallel.download](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.parallel.download.html)

Let's install it:

In [ ]:
gs.run_command("g.extension", extension="i.sentinel")

Check if the module is there by running it with optional arguments:

In [ ]:
gs.core.find_program("i.sentinel.download", "--help")

### Sentinel 2 data search and download

[Sentinel-2 L2A products](https://sentinels.copernicus.eu/de/web/sentinel/user-guides/sentinel-2-msi/product-types/level-2a) will be used to avoid the need of computing atmospheric corrections. 

Let’s search for the latest available product by means of [i.sentinel.download](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.download.html). Setting the `-l` flag, the result will only
be printed. The download procedure will be demonstrated later. 

In order to search and download Sentinel products from the Copernicus Open Access Hub, you have to create an account first (see below). See the manual page of [i.sentinel.download](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.download.html) module for details. Upload or create a new text
file in the data directory (`homedir`) named `esa_credentials.txt` containing two lines: username and password.

#### Note

To get username and password you need to register at the [Copernicus Open Access Hub](https://scihub.copernicus.eu/), see the [register new account](https://scihub.copernicus.eu/dhus/#/self-registration) page for signing up. Once you registered, create a text file named `esa_credentials.txt` 
with the following content:
```
    username
    password
```
and move it within your working directory

In [ ]:
# list available Sentinel-2 L2A scenes for AOI
# note that we use parse_command() in order to intercept the output
gs.parse_command("i.sentinel.download", 
                 flags="l", 
                 producttype="S2MSI2A",
                 map="urban_area_raleigh",
                 settings=os.path.join(homedir, "esa_credentials.txt"))

By default, the module returns all the products meeting the defined criteria for the last 60 days. Let’s change
the search period setting `start` and `end` options. We will also limit products by `clouds` coverage percentage 
threshold and `sort` them by ingestion date.

In [ ]:
gs.parse_command("i.sentinel.download", 
                 flags="l", 
                 producttype="S2MSI2A", 
                 map="urban_area_raleigh",
                 settings=os.path.join(homedir, "esa_credentials.txt"),
                 start="2022-02-01", 
                 end="2022-05-31", 
                 clouds="5",
                 sort="ingestiondate",
                 limit=10)

<div class="alert alert-info"> If a long list of products have been found, you can limit the amount with the <code>limit</code> option as we did above.

Let's save the output of the search into a list and then beautify the display by creating a pandas table.

In [ ]:
list_prod = gs.read_command("i.sentinel.download", 
                            flags="l", 
                            producttype="S2MSI2A", 
                            map="urban_area_raleigh",
                            settings=os.path.join(homedir, "esa_credentials.txt"), 
                            footprints="s2_footprints", # we save the footprints in a vector file
                            start="2022-02-01", 
                            end="2022-05-31", 
                            clouds="5",
                            sort="ingestiondate",
                            limit=10)

In [ ]:
# print plain list
list_prod

In [ ]:
import pandas as pd
from io import StringIO

pd.read_csv(StringIO(list_prod), delimiter=" ", usecols=[0, 1, 2, 4, 5, 6, 7],
            names=['uuid', 'scene', 'date', 'cloud', 'product', 'size', 'unit'])

In [ ]:
# list available vector maps in sentinel2
gs.list_grouped(type="vector")["sentinel2"]

In [ ]:
# diplay footprints (you may want to zoom out a bit)
fp_map = gj.InteractiveMap(width = 400, use_region=True, tiles="OpenStreetMap")
fp_map.add_vector("s2_footprints")
fp_map.add_vector("urban_area_raleigh")
fp_map.add_layer_control(position = "bottomright")
fp_map.show()

The next step is to download the scene or scenes of interest. Just remove the `-l` flag and add the `output` option in order to define the path to the output directory where data should be saved. 

As download might take quite some time, we'll **skip this part** and directly use an already prepared set of smaller, ready to import scenes which we downloaded above. Still, we leave an example below for future reference :)

Go to section **"Importing Sentinel 2 data"**

In [ ]:
# Example: download of a selected scene (2022-06-17, T15:58:29Z)
# gs.run_command("i.sentinel.download", 
#               settings=s2_credentials, 
#               uuid="cfa30609-5627-4788-b7ff-768e2df99975", 
#               output=s2_data)

### Importing Sentinel-2 data

Before importing or linking Sentinel-2 data we print a list of filtered raster files including projection match 
(1 for match, otherwise 0). If the CRS of the input data differs from that of the current location, you should 
consider reprojection (`-r`flag) or creating a new location for import.

*Important*: Data will be imported into the new location by means of the [i.sentinel.import](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.import.html) tool. 
The command will, by default, import **all** Sentinel bands from `input` directory recursively. 
Before importing the data, let’s check content of the input directory by means of the `-p` flag.

In [ ]:
# Check list of pre-downloaded Sentinel-2 scenes, with i.sentinel.import (-p: print)
gs.parse_command("i.sentinel.import", 
                 flags="p", 
                 input=s2_data)

To speed up things, we'll limit the S2 data import to the RGB and NIR bands (2, 3, 4, 8A) in 10 m spatial resolution using the `pattern` option. Let's first print the bands that will be imported:

In [ ]:
# print only to test band selection
gs.parse_command("i.sentinel.import", 
                 flags="p", 
                 input=s2_data, 
                 pattern="B(02|03|04|08)_10m")

By default, input data are imported into GRASS and converted into GRASS native data format.
Alternatively, data can be linked if the `-l` flag is provided. It is also
useful to import cloud mask vector features (`-c` flag). In addition, we'll use the 
`register_output` option to produce a timestamp plain text file
which will be used later on to create a time series.

In [ ]:
# for S2 import, allow for using 2GB of RAM for faster operations.
# (s2_data and s2_timestamps are defined above)
# this takes up to a few minutes...
gs.parse_command("i.sentinel.import", 
                 flags="rcsj", 
                 input=s2_data, 
                 pattern="B(02|03|04|08)_10m", 
                 memory=4000, 
                 extent="input",
                 register_output=s2_timestamps)

In [ ]:
# list selected raster maps
gs.list_grouped(type="raster")['sentinel2']

In [ ]:
# check metadata of one of the imported bands
gs.raster_info(map="T17SQV_20220617T155829_B03_10m")["comments"]

In [ ]:
# print timestamp file for inspection
with open(s2_timestamps, 'r') as f:
    content = f.read()
    print(content)
    f.close()

<div class="alert alert-success">
<b>Semantic labels</b><br>
A fairly new concept within GRASS GIS is semantic labels. These are especially relevant for satellite imagery as they allow us to identify to which sensor and band a given raster corresponds to. These labels are particularly relevant when working with satellite image collections and also when classifying different scenes. We will see it later, but by generating a spectral signature for a given set of bands, it can be re-used to classify another scene as long as the semantic labels are the same. Be ware – although it is possible to re-use spectral signatures to any scene with the same bands, temporal changes (seasons, weather impact) limit their applicability only to scenes obtained more or less at the same time.

### Displaying maps with `grass.jupyter` functions

In [ ]:
# create Map instance
b3_map = gj.Map(width=400)
# add a raster, vector and legend to the map
b3_map.d_rast(map="T17SQV_20220617T155829_B03_10m")
b3_map.d_vect(map="lakes")
b3_map.d_legend(raster="T17SQV_20220617T155829_B03_10m", 
                title="Reflectance", 
                fontsize=10, at=(70, 93, 80, 90), flags="b")
b3_map.d_barscale()
# display map
b3_map.show()

In [ ]:
# set color table of bands 4, 3 and 2 to grey
gs.run_command("r.colors", 
               map="T17SQV_20220617T155829_B04_10m,T17SQV_20220617T155829_B03_10m,T17SQV_20220617T155829_B02_10m", 
               color="grey")

In [ ]:
# color enhancing for RGB composition
gs.run_command("i.colors.enhance", 
               red="T17SQV_20220617T155829_B04_10m",
               green="T17SQV_20220617T155829_B03_10m", 
               blue="T17SQV_20220617T155829_B02_10m",
               strength=92)

In [ ]:
# set region to "elevation" map and align to the S2 data
gs.run_command("g.region", 
               raster="elevation",
               align="T17SQV_20220617T155829_B04_10m",
               flags="p")

In [ ]:
# display the enhanced RGB combination
rgb = gj.Map(width=400, use_region=True)
rgb.d_rgb(red="T17SQV_20220617T155829_B04_10m",
          green="T17SQV_20220617T155829_B03_10m", 
          blue="T17SQV_20220617T155829_B02_10m")
rgb.show()

## 8. Spectral indices of vegetation and water

We will use i.vi and i.wi (addon) to estimate NDVI and NDWI vegetation and water indices. See [i.vi](https://grass.osgeo.org/grass-stable/manuals/i.vi.html) and [i.wi](https://grass.osgeo.org/grass-stable/manuals/addons/i.wi.html) for more other available indices.

In [ ]:
# estimate vegetation indices
gs.run_command("i.vi", 
               red="T17SQV_20220528T155819_B04_10m", 
               nir="T17SQV_20220528T155819_B08_10m", 
               output="T17SQV_20220528T155819_NDVI_10m", 
               viname="ndvi")

# add semantic label
gs.run_command("r.support", 
               map="T17SQV_20220528T155819_NDVI_10m", 
               semantic_label="S2_NDVI")

In [ ]:
# install extension
gs.run_command("g.extension", extension="i.wi")

In [ ]:
# estimate water indices
gs.run_command("i.wi", 
               green="T17SQV_20220528T155819_B03_10m", 
               nir="T17SQV_20220528T155819_B08_10m", 
               output="T17SQV_20220528T155819_NDWI_10m", 
               winame="ndwi_mf")

# set ndwi color palette
gs.run_command("r.colors", map="T17SQV_20220528T155819_NDWI_10m", color="ndwi")

# add semantic label
gs.run_command("r.support", 
               map="T17SQV_20220528T155819_NDWI_10m", 
               semantic_label="S2_NDWI")

In [ ]:
# check metadata of NDVI
gs.raster_info(map="T17SQV_20220528T155819_NDVI_10m")

In [ ]:
# interactive maps
idx_map = gj.InteractiveMap(width = 400, use_region=True, tiles="OpenStreetMap")
idx_map.add_raster("T17SQV_20220528T155819_NDVI_10m", opacity=0.7)
idx_map.add_raster("T17SQV_20220528T155819_NDWI_10m", opacity=0.7)
idx_map.add_layer_control(position = "bottomright")
idx_map.show()
# ... use the layer selector in the corner to enable/disable the NDVI/NDWI layers

#### GRASS GIS maps as numpy arrays

GRASS maps can be read as numpy arrays thanks to the array function of the grass.script library. This facilitates many operations with python libraries that require an array as input. In this case, we demonstrate its use plotting an histogram.

In [ ]:
# Import required libraries
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from grass.script import array as garray

# Read NDVI as numpy array
ndvi = garray.array(mapname="T17SQV_20220528T155819_NDVI_10m", null="nan")
ndwi = garray.array(mapname="T17SQV_20220528T155819_NDWI_10m", null="nan")
print(ndvi.shape,ndwi.shape)

In [ ]:
# Plot NDVI and NDWI
sns.set_style('darkgrid')
fig, axs = plt.subplots(1, 2, figsize=(7, 7))
sns.histplot(ax=axs[0], data=ndvi.ravel(), kde=True, color="olive")
sns.histplot(ax=axs[1], data=ndwi.ravel(), kde=True, color="skyblue")
plt.show()

## 10. NDVI time series data processing

### A few concepts of time series data processing in GRASS GIS

GRASS GIS offers specialized tools for spatio-temporal data
processing, see GRASS documentation [temporalintro](https://grass.osgeo.org/grass-stable/manuals/temporalintro.html) for details and the [temporal data processing](https://grasswiki.osgeo.org/wiki/Temporal_data_processing) wiki for examples and a workflow tutorial.

GRASS introduces three special data types that are designed to handle time-series:

* *Space-time raster datasets* (`strds`) for managing raster map time series.

* *Space-time 3D raster datasets* (`str3ds`) for managing 3D raster map time series.

* *Space-time vector datasets* (`stvds`) for managing vector map time series.
  
<a href="assets/img/tgrass_flowchart.png">
  <img src="assets/img/tgrass_flowchart.png"
   alt="TGRASS flowchart"
   title="GRASS flowchart"
   width="65%">
</a>


### Create space-time dataset

At this moment a new space-time dataset can be created by means of [t.create](https://grass.osgeo.org/grass-stable/manuals/t.create.html) and all imported Sentinel bands registered with [t.register](https://grass.osgeo.org/grass-stable/manuals/t.register.html) and the timestamps file we created when we imported S2 bands.

In [ ]:
gs.run_command("t.create", 
               output="s2_nc", 
               title="Sentinel L2A - North Carolina", 
               desc="Tile T17SQV - 2022")

gs.run_command("t.register", 
               input="s2_nc", 
               file=s2_timestamps)

Let’s check basic metadata with [t.info](https://grass.osgeo.org/grass-stable/manuals/t.info.html) and list the registered maps with [t.rast.list](https://grass.osgeo.org/grass-stable/manuals/t.rast.list.html).

In [ ]:
# Print time series info
print(gs.read_command("t.info", input="s2_nc"))

In [ ]:
# List registered bands in the space-time cube
print(gs.read_command("t.rast.list", 
                      input="s2_nc", 
                      columns="name,start_time,semantic_label"))

We'll now use a special syntaxis to list only band 4 raster maps withing the time series:

In [ ]:
# List only band 4 maps
print(gs.read_command("t.rast.list", 
                      input="s2_nc.S2_4", 
                      columns="name,start_time,semantic_label"))

### NDVI Space-Time computation

For NDVI computation the 4th and 8th bands are required, as we saw above for a single map. 
Now, we will create a time series of NDVI maps. We will take advantage of the semantic labels syntax and use
[t.rast.mapcalc](https://grass.osgeo.org/grass-stable/manuals/t.rast.mapcalc.html) to estimate NDVI for all the timestamps in the time series, using band 4 and 8 subsets.

In [ ]:
gs.run_command("t.rast.mapcalc", 
               inputs="s2_nc.S2_8,s2_nc.S2_4", 
               output="s2_ndvi", 
               basename="s2_ndvi",
               expression="float(s2_nc.S2_8 - s2_nc.S2_4) / (s2_nc.S2_8 + s2_nc.S2_4)")

When computation is finished, the *ndvi* color table can be set with [t.rast.colors](https://grass.osgeo.org/grass-stable/manuals/t.rast.colors.html):

In [ ]:
gs.run_command("t.rast.colors", input="s2_ndvi", color="ndvi")

In [ ]:
print(gs.read_command("t.info", input="s2_ndvi"))

### Time series plots

Let’s check content of the new dataset by means of [t.rast.list](https://grass.osgeo.org/grass-stable/manuals/t.rast.list.html):

In [ ]:
print(gs.read_command("t.rast.list", 
                      input="s2_ndvi", 
                      columns="name,start_time,min,max"))

If we save the previous output to a file, we can then plot the min and max time series:

In [ ]:
gs.run_command("t.rast.list", 
                input="s2_ndvi", 
                columns="name,start_time,min,max",
                format="csv",
                separator="comma",
                output=os.path.join(homedir,"ndvi.csv"))

In [ ]:
# Read the csv and plot
ndvi = pd.read_csv(os.path.join(homedir,"ndvi.csv"))
ndvi.plot(0, [2,3], subplots=False)

We could also use [t.rast.univar](https://grass.osgeo.org/grass-stable/manuals/t.rast.univar.html) to obtain extended statistics:

In [ ]:
# Get extended univar stats and save them as a csv file
gs.run_command("t.rast.univar",
                flags="e",
                input="s2_ndvi",
                output=os.path.join(homedir,"ndvi_ext_stats.csv"),
                separator="comma")

In [ ]:
# Read the csv and plot
ndvi = pd.read_csv(os.path.join(homedir,"ndvi_ext_stats.csv"))
ndvi['start'] = pd.to_datetime(ndvi.start, format="%Y-%m-%d", exact=False)
ndvi.plot.line(1, [3,4,5], subplots=False)

### Query time series in a single point

`g.region` command allows us to get the coordinates of the center of the computational region, we'll use those to query the NDVI time series.

In [ ]:
# Get region center coordinates for query (center_easting, center_northing values)
gs.region(complete=True)

In [ ]:
# Query map at center coordinates
print(gs.read_command("t.rast.what", 
                      strds="s2_ndvi", 
                      coordinates="637500,221750", 
                      layout="col", 
                      flags="n"))

### Time series animation

Note: [TimeSeriesMap()](https://grass.osgeo.org/grass-stable/manuals/libpython/grass.jupyter.html?highlight=timeseriesmap#module-grass.jupyter.timeseriesmap) of `grass.jupyter` is still experimental and under development.

In [ ]:
### YET TO BE SKIPPED - in GRASS GIS 8.2.0 it takes "forever", bugfix pending.

## reduce resolution for faster display of time series, save original first for later
#gs.parse_command("g.region", save="default_res")
#gs.parse_command("g.region", flags="pa", res=50)
 
## Display newly created NDVI time series map
#ndviseries = gj.TimeSeriesMap(use_region=True)
#ndviseries.add_raster_series("s2_ndvi", fill_gaps=False)
#ndviseries.d_legend(color="black", at=(10,40,2,6))
#ndviseries.d_barscale()
#ndviseries.show()  # Create TimeSlider

# optionally, write out to animated GIF
# ndviseries.save("image.gif")

In [ ]:
## restore original region
#gs.parse_command("g.region", region="default_res")

## 11. Creating an image stack (imagery group)

**Stack of maps = imagery group**

When you work with a stack of raster maps (e.g., R-G-B channels or more) in GRASS GIS, you can best handle this stack by creating a raster group with [i.group](https://grass.osgeo.org/grass-stable/manuals/i.group.html). It is just based on metadata, so it does not take up more disk space.

In [ ]:
# Since imagery groups can not be overwritten, 
# we delete a potentially leftover "s2" group from a previous run
gs.run_command("g.remove", 
               type="group", 
               name="s2", 
               flags="f")

In [ ]:
# Generate list of selected S2 maps
s2_maps = gs.list_grouped(type="raster", pattern="*20220528T155819*")['sentinel2']
print(s2_maps)

In [ ]:
# Create group and subgroup with S2 bands
gs.run_command("i.group", group="s2", subgroup="s2", input=s2_maps)
print(gs.read_command("i.group", group="s2", flags="l"))

## 12. Object recognition with image segmentation

We'll use [i.segment](https://grass.osgeo.org/grass-stable/manuals/i.segment.html) to perform image segmentation. The resulting map will be used together with S2 bands, NDVI and NDWI to perform supervised classification.

In [ ]:
# Threshold = 0 merges only identical segments; threshold = 1 merges all
gs.run_command("i.segment", 
               group="s2", 
               threshold="0.05", 
               minsize="100", 
               output="sentinel_segments_min100", 
               goodness="sentinel_segments_goodness_min100",
               memory=2000)

In [ ]:
# Display newly created segments raster map
segments = gj.InteractiveMap(width = 400, use_region=True)
segments.add_raster("sentinel_segments_min100", opacity=0.3)
segments.add_raster("s2_ndvi_4", opacity=0.3)
segments.add_layer_control(position = "bottomright")
segments.show()

In [ ]:
# Show univariate statistics of goodness-of-fit raster map, with extended statistics (quartiles)
print(gs.read_command("r.univar",
                      map="sentinel_segments_goodness_min100", 
                      flags="ge"))

In [ ]:
# Assign color table (low fit values: blue; high fit values: green)
gs.run_command("r.colors", 
               map="sentinel_segments_goodness_min100", 
               color="byg", 
               flags="e")

In [ ]:
# Display newly created goodness-of-fit raster map
segments = gj.InteractiveMap(width = 400, use_region=True)
segments.add_raster("sentinel_segments_goodness_min100", opacity=0.8)
segments.add_vector("urban_area_raleigh")
segments.add_layer_control(position = "bottomright")
segments.show()

## 13. Supervised Classification: RandomForest

We will now demonstrate a very much simplified workflow to perform a supervised [Random Forest classification](https://en.wikipedia.org/wiki/Random_forest).

We will feed the following data into the model:

- NDVI and NDWI maps (created above)
- image segmentation (created above)
- random training points extracted from landuse map

First we inspect the raster maps available in the current mapset (i.e., `sentinel2`), just to recall their names.

In [ ]:
gs.list_grouped(type="raster")["sentinel2"]

### Creation of a classification training map by sampling from existing data

In order to generate training data for the Sentinel-2 image classification, we will use the [National Land Cover Database (NLCD) 2019](https://www.lib.ncsu.edu/gis/lulc). It is available for download (30m raster map) from [here](https://drive.google.com/open?id=18D99kuotQp_BkxBnkn8OS3qgCeLVwovb&authuser=0). However, we have already prepared the dataset (the `nc_nlcd2019` landuse map). We will use it to perform stratified sampling to retrieve training data.

In [ ]:
# Check raster categories of landuse map
print(gs.read_command("r.category", 
                      map="nc_nlcd2019", 
                      separator="comma"))

In [ ]:
# display nc_nlcd2019 landuse raster map
lulc = gj.InteractiveMap(width = 400, use_region=True, tiles="OpenStreetMap")
lulc.add_raster("nc_nlcd2019", opacity=0.6)
lulc.add_layer_control(position = "bottomright")
lulc.show()

We already note differences between the underlying OpenStreetMap data and the 30m NLCD map.

In [ ]:
# show simple legend
legend = gj.Map(width=400, use_region=True)
# at=bottom,top,left,right, percentage of screen coordinates (0,0 is lower left)
legend.d_legend(raster="nc_nlcd2019", 
                title="Classes",
                fontsize=10, at=(10, 90, 50, 90), 
                flags="n")
legend.show()

### Random sampling from rasterized simplified landuse map

We now perform stratified sampling, i.e. we extract for each land use class `n` sampling points, using the GRASS GIS addon [r.sample.category](https://grass.osgeo.org/grass-stable/manuals/addons/r.sample.category.html).

First, we install this addon.

In [ ]:
gs.run_command("g.extension", extension="r.sample.category")

In [ ]:
# Stratified random sampling, generated vector points
gs.run_command("r.sample.category", 
               input="nc_nlcd2019", 
               output="landuse_train", 
               n="100")

In [ ]:
# display newly created vector points map
train = gj.InteractiveMap(width = 400, use_region=True)
train.add_raster("nc_nlcd2019", opacity=0.7)
train.add_vector("landuse_train")
train.add_layer_control(position = "bottomright")
train.show()

In [ ]:
# List column names of vector points map
gs.vector_columns("landuse_train", 
                  getDict=False)

In [ ]:
# Show vector attribute table
gs.vector_db_select("landuse_train")

In [ ]:
# Check column data types
print(gs.read_command("v.info", map="landuse_train", flags="c"))

Since the machine learning classifier expects raster points as input, we convert the vector sampling points accordingly using  [v.to.rast](https://grass.osgeo.org/grass-stable/manuals/v.to.rast.html).

In [ ]:
# Convert points from vector to raster model
gs.run_command("v.to.rast", 
               input="landuse_train", 
               output="landuse_train", 
               use="attr", 
               attribute_column="nc_nlcd2019", 
               label_column="label")

In [ ]:
# Check raster categories of new raster training map
# Skip reporting on empty cells
print(gs.read_command("r.report", 
                      map="landuse_train",
                      flags="n"))

In [ ]:
# Display newly created raster map - zoom in to better spot the raster sampling points
train = gj.InteractiveMap(width = 400, use_region=True)
train.add_raster("landuse_train", opacity=0.8)
train.add_layer_control(position = "bottomright")
train.show()

### Perform machine learning model training (RandomForest)

First we have to install the [r.learn.ml2](https://grass.osgeo.org/grass-stable/manuals/addons/r.learn.ml2.html) extention. It consists of two modules: `r.learn.train` and `r.learn.predict`.

In [ ]:
# Install ML extension
gs.run_command("g.extension", extension="r.learn.ml2")

In [ ]:
# Add segmentation map created above to group and subgroup already populated with S2 bands, NDWI and NDVI
gs.run_command("i.group", 
               group="s2", 
               subgroup="s2", 
               input="sentinel_segments_min100")

# List group content
print(gs.read_command("i.group", group="s2", flags="l"))

We now train the ML model using [r.learn.train](https://grass.osgeo.org/grass-stable/manuals/addons/r.learn.train.html), with model "RandomForestClassifier".

In [ ]:
# Train a random forest classification model using r.learn.train
gs.run_command("r.learn.train", 
               group="s2", 
               training_map="landuse_train",
               model_name="RandomForestClassifier",
               n_estimators="500", 
               save_model=os.path.join(homedir, "rf_model.gz"))

 The model has been stored in the file `rf_model.gz` for use in the prediction step of the supervised classification.

In [ ]:
os.listdir(homedir)

### Perform ML supervised classification

The trained model will now be applied to the entire dataset.

In [ ]:
# Perform prediction using r.learn.predict
gs.run_command("r.learn.predict", 
               group="s2", 
               load_model=os.path.join(homedir, "rf_model.gz"), 
               output="sentinel_rf")

In [ ]:
# Set color table, we transfer the colors from the original landuse map
gs.run_command("r.colors", map="sentinel_rf", raster="nc_nlcd2019")

With this, the (oversimplified) supervised classification has been completed and we can display the result.

### Reporting and display

In [ ]:
# Display newly created sentinel_rf map
rfmap = gj.InteractiveMap(width = 600, tiles="OpenStreetMap")
rfmap.add_raster("sentinel_rf", opacity=0.7)
# rfmap.add_raster("nc_nlcd2019", opacity=0.7)
rfmap.add_layer_control(position = "bottomright")
rfmap.show()

In [ ]:
# Show legend
legend = gj.Map(width=400, use_region=True)
legend.d_legend(raster="sentinel_rf", 
                title="Classes",
                fontsize=14, 
                at=(10, 80, 10, 40), 
                flags="n")
legend.show()

In [ ]:
# Show class distribution in percent
print(gs.read_command("r.report", 
                      map="sentinel_rf", 
                      units="p", 
                      flags="h"))

In [ ]:
# export map to COG
gs.run_command("r.out.gdal", 
               flags="fmt", #
               input="sentinel_rf", 
               output=os.path.join(homedir, "nc_sentinel2_RF.tif"),
               format="COG", 
               overviews="4")

Keep in mind, this classification was just a simplified example to show how the procedure works.

At this moment you should use [r.kappa](https://grass.osgeo.org/grass-stable/manuals/r.kappa.html) to calculate accuracy of classification. As this step would require either field observation data or manual interpretation of the scene, we'll leave this as an exercise to do at home.

In [ ]:
# Open the tif in QGIS, adapt path accordingly
!qgis $homedir/nc_sentinel2_RF.tif

## 14. Supervised Classification: Maximum Likelihood

We will now demonstrate the workflow to perform a supervised maximum likelihood classification which is integrated with the semantic labels metadata class, and hence allow us to use the same spectral signature to classify multiple scenes as long as the raster map order in the group is the same.

Let's first check the semantic labels of the bands in our `s2` group:

In [ ]:
band_list = gs.read_command("i.group", group="s2", flags="lg")

In [ ]:
# Add semantic label to the segmentation
gs.run_command("r.support", 
               map="sentinel_segments_min100", 
               semantic_label="S2_seg")

In [ ]:
for m in band_list.split():
    sl = gs.raster_info(m)['semantic_label']
    print(m,sl)

Now, we generate the signature file based on the training sample that we obtained earlier, this will then be the input for the maximum likelihood classification

In [ ]:
# obtain signature files
gs.run_command("i.gensig", 
               trainingmap="landuse_train", 
               group="s2", 
               subgroup="s2", 
               signaturefile="sig_sentinel")

In [ ]:
# perform ML supervised classification
gs.run_command("i.maxlik", 
               group="s2", 
               subgroup="s2", 
               signaturefile="sig_sentinel", 
               output="sentinel_maxlik")

In [ ]:
# check classes
print(gs.read_command("r.category", 
                      map="sentinel_maxlik", 
                      separator="comma"))

In GRASS 8.2+, [i.maxlik](https://grass.osgeo.org/grass-stable/manuals/i.maxlik.html) classifier does not preserve the original class values in the output. Thus, here is a lookup-table for original class numbers and new category values:

class|nlcd_class|landuse|RGB
--- | --- | --- | --- 
1|11|Open Water|072:109:162
2|21|Developed, Open Space|225:205:206
3|22|Developed, Low Intensity|220:152:129
4|23|Developed, Medium Intensity|241:001:000
5|24|Developed, High Intensity|171:001:001
6|41|Deciduous Forest|108:169:102
7|42|Evergreen Forest|029:101:051
8|43|Mixed Forest|189:204:147
9|81|Hay/Pasture|221:216:062
10|90|Woody Wetlands|187:215:237

In [ ]:
# Set color table
colours = ["1 072:109:162", "2 225:205:206", "3 220:152:129", "4 241:001:000", "5 171:001:001", "6 108:169:102", "7 029:101:051", "8 189:204:147", "9 221:216:062", "10 187:215:237"]
gs.write_command("r.colors", map="sentinel_maxlik", rules="-", stdin="\n".join(colours))

In [ ]:
# display results
maxlik_sup_class = gj.Map(width=500, use_region=True)
maxlik_sup_class.d_rast(map="sentinel_maxlik")
maxlik_sup_class.d_legend(raster="sentinel_maxlik", 
                          title="Class", 
                          fontsize=12, 
                          at=(70, 95, 75, 90), 
                          flags="bn")
maxlik_sup_class.d_barscale()
maxlik_sup_class.show()

In [ ]:
# percentage of each class
print(gs.read_command("r.report", 
                      map="sentinel_maxlik", 
                      units="p", 
                      flags="h"))

In [ ]:
# class statistics: NDVI
class_stats = gs.read_command("r.univar", 
                              map="T17SQV_20220528T155819_NDVI_10m", 
                              zones="sentinel_maxlik", 
                              flags="t")

In [ ]:
pd.read_csv(StringIO(class_stats), 
            delimiter="|", 
            usecols=[1, 4, 5, 7])

Next, and to demonstrate the use of semantic labels, we will classify another sentinel scene with the same signature obtained earlier. To this aim, we need to:
1. create a new imagery group for a different scene with the exact same band order
1. estimate NDVI and NDWI and assign semantic labels
1. run a segmentation and assign semantic labels
1. check group and semantic labels
1. run `i.maxlik`

<div class="alert alert-warning">
Be ware – changes over time (phenology, weather) will make spectral signatures to not fit well or at all. Do not use same signatures for a different season!

In [ ]:
s2_maps = gs.list_grouped(type="raster", pattern="*20220617*")['sentinel2']
s2_maps

In [ ]:
# Since imagery groups can not be overwritten, 
# we delete any leftover "s2_new" group from previous runs
gs.run_command("g.remove", 
               type="group", 
               name="s2_new", 
               flags="f")

In [ ]:
gs.run_command("i.group", group="s2_new", subgroup="s2_new", input=s2_maps)
print(gs.read_command("i.group", group="s2_new", flags="l"))

In [ ]:
# estimate NDVI
gs.run_command("i.vi", 
               red="T17SQV_20220617T155829_B04_10m", 
               nir="T17SQV_20220617T155829_B08_10m", 
               output="T17SQV_20220617T155829_NDVI_10m", 
               viname="ndvi")

# add semantic label
gs.run_command("r.support", 
               map="T17SQV_20220617T155829_NDVI_10m", 
               semantic_label="S2_NDVI")

In [ ]:
# estimate NDWI
gs.run_command("i.wi", 
               green="T17SQV_20220617T155829_B03_10m", 
               nir="T17SQV_20220617T155829_B08_10m", 
               output="T17SQV_20220617T155829_NDWI_10m", 
               winame="ndwi_mf")

# add semantic label
gs.run_command("r.support", 
               map="T17SQV_20220617T155829_NDWI_10m", 
               semantic_label="S2_NDWI")

In [ ]:
# add NDVI and NDWI to s2_mew group
gs.run_command("i.group", 
               group="s2_new", 
               subgroup="s2_new", 
               input="T17SQV_20220617T155829_NDVI_10m,T17SQV_20220617T155829_NDWI_10m")

# print maps in the group
print(gs.read_command("i.group", group="s2_new", flags="l"))

In [ ]:
# Run segmentation
gs.run_command("i.segment", 
               group="s2_new", 
               threshold="0.05", 
               minsize="100", 
               output="sentinel_new_segments_min100", 
               goodness="sentinel_new_segments_goodness_min100")

In [ ]:
# Add semantic label to the segmentation
gs.run_command("r.support", 
               map="sentinel_new_segments_min100", 
               semantic_label="S2_seg")

In [ ]:
# Add segmentation to the s2_new group
gs.parse_command("i.group", group="s2_new", subgroup="s2_new", input="sentinel_new_segments_min100")

In [ ]:
# Check
print(gs.read_command("i.group", group="s2_new", flags="l"))

In [ ]:
# Run the classification
gs.run_command("i.maxlik", 
               group="s2_new", 
               subgroup="s2_new", 
               signaturefile="sig_sentinel", 
               output="sentinel_maxlik_new")

In [ ]:
# Set color table
colours = ["1 072:109:162", "2 225:205:206", "3 220:152:129", "4 241:001:000", "5 171:001:001", "6 108:169:102", "7 029:101:051", "8 189:204:147", "9 221:216:062", "10 187:215:237"]
gs.write_command("r.colors", map="sentinel_maxlik_new", rules="-", stdin="\n".join(colours))

In [ ]:
# display results
maxlik_sup_class = gj.Map(width=500, use_region=True)
maxlik_sup_class.d_rast(map="sentinel_maxlik_new")
maxlik_sup_class.d_legend(raster="sentinel_maxlik_new", 
                          title="Class", 
                          fontsize=12, 
                          at=(60, 95, 70, 90), 
                          flags="bn")
maxlik_sup_class.d_barscale()
maxlik_sup_class.show()

## 15. What's next?

You may enjoy more Jupyter notebooks at: https://github.com/OSGeo/grass/tree/main/doc/notebooks

### Talk to us

- Veronica Andreo, PhD, https://veroandreo.gitlab.io/
- Markus Neteler, PhD, https://www.mundialis.de/en/neteler/
- Māris Nartišs, PhD

### References

- [GRASS GIS 8.2.0 Reference Manual](https://grass.osgeo.org/grass-stable/manuals/)
- [GRASS GIS Addons Reference Manuals](https://grass.osgeo.org/grass-stable/manuals/addons/)
- [GRASS GIS Python library documentation](https://grass.osgeo.org/grass-stable/manuals/libpython/)
- [Unleash the power of GRASS GIS with Jupyter](https://github.com/ncsu-geoforall-lab/grass-gis-workshop-foss4g-2022)
- List of [Tutorials](https://grass.osgeo.org/learn/tutorials/) at the GRASS GIS website